# Bond Option Pricing using Various Models

Value an option on a coupon paying bond using the Hull-White, Black-Derman-Toy and Black-Karasinski model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.bk_tree import BKTree
from financepy.models.bdt_tree import BDTTree
from financepy.models.hw_tree import HWTree
from financepy.products.bonds import *

## Set up the Discount Curve

In [3]:
settle_dt = Date(1, 12, 2019)

Set up discount curve

In [4]:
rate = 0.10
dc_type = DayCountTypes.THIRTY_360_BOND
fixed_freq = FrequencyTypes.SEMI_ANNUAL
discount_curve = DiscountCurveFlat(settle_dt, rate, fixed_freq, dc_type)

## Set up the Bond Option

First create the bond

In [5]:
issue_dt = Date(1,12,2018)
maturity_dt = issue_dt.add_tenor("3Y")
coupon = 0.10
freq_type = FrequencyTypes.SEMI_ANNUAL
dc_type = DayCountTypes.THIRTY_360_BOND
bond = Bond(issue_dt, maturity_dt, coupon, freq_type, dc_type)

Let's first price the bond on the libor curve

In [6]:
cp = bond.clean_price_from_discount_curve(settle_dt, discount_curve)
dp = bond.dirty_price_from_discount_curve(settle_dt, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Dirty Price:   100.000


In [7]:
settle_dt

01-DEC-2019

In [8]:
bond.print_payments(settle_dt)

 01-JUN-2020      5.00000 
 01-DEC-2020      5.00000 
 01-JUN-2021      5.00000 
 01-DEC-2021    105.00000 



# Create the Bond Options

Then define and create the option

In [9]:
expiry_dt = settle_dt.add_tenor("18m")
strike_price = 95.0
face_amount = 100.0

In [10]:
europeanCallBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.EUROPEAN_CALL)
americanCallBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.AMERICAN_CALL)

In [11]:
europeanPutBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.EUROPEAN_PUT)
americanPutBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [12]:
cp = bond.clean_price_from_discount_curve(expiry_dt, discount_curve)
dp = bond.dirty_price_from_discount_curve(expiry_dt, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Dirty Price:   100.000


In [13]:
df = discount_curve.df(expiry_dt)
call_intrinsic = max(cp - strike_price, 0) * df
put_intrinsic = max(strike_price - cp, 0) * df

In [14]:
call_intrinsic

np.float64(4.319187992657368)

In [15]:
put_intrinsic

np.float64(0.0)

## Tree Models

Set the model parameters, start with zero vol.

In [16]:
num_steps = 200

In [17]:
sigma = 0.00001
a = 0.1
modelHW = HWTree(sigma*rate, a, num_steps)
modelBK = BKTree(sigma, a, num_steps)
modelBDT = BDTTree(sigma, num_steps)

Check some limits - the call option should be worth the intrinsic.

In [18]:
ec = europeanCallBondOption.value(settle_dt, discount_curve, modelHW)
ac = americanCallBondOption.value(settle_dt, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.30649
American Call Value:   4.96842


The put option too.

In [19]:
ep = europeanPutBondOption.value(settle_dt, discount_curve, modelHW)
ap = americanPutBondOption.value(settle_dt, discount_curve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00000


Now we turn on the volatility.

In [20]:
sigma = 0.20
a = 0.1
modelHW = HWTree(sigma*rate, a, num_steps)
modelBK = BKTree(sigma, a, num_steps)
modelBDT = BDTTree(sigma, num_steps)

In [21]:
ec = europeanCallBondOption.value(settle_dt, discount_curve, modelHW)
ac = americanCallBondOption.value(settle_dt, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.30650
American Call Value:   5.26581


In [22]:
ep = europeanPutBondOption.value(settle_dt, discount_curve, modelHW)
ap = americanPutBondOption.value(settle_dt, discount_curve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00454


Copyright (c) Dominic O'Kane 2020